In [ ]:
!pip install gensim

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import spacy

In [ ]:
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# base reviews
df = pd.read_csv('/content/BA_AirlineReviews.csv')

In [ ]:
# Base de interesse: comentátios (texto) e recomendação ('yes', 'no').
base = df[['ReviewBody', 'Recommended']]

In [ ]:
base.head()

,ReviewBody,Recommended
0,4 Hours before takeoff we received a Mail stat...,no
1,I recently had a delay on British Airways from...,no
2,"Boarded on time, but it took ages to get to th...",yes
3,"5 days before the flight, we were advised by B...",no
4,"We traveled to Lisbon for our dream vacation, ...",no


# Stop words e tokens

In [ ]:
from gensim.parsing.preprocessing import remove_stopwords

In [ ]:
base.ReviewBody[0], remove_stopwords(base.ReviewBody[0])

('4 Hours before takeoff we received a Mail stating a cryptic message that there are disruptions to be expected as there is a limit on how many planes can leave at the same time. So did the capacity of the Heathrow Airport really hit British Airways by surprise, 4h before departure? Anyhow - we took the one hour delay so what - but then we have been forced to check in our Hand luggage. I travel only with hand luggage to avoid waiting for the ultra slow processing of the checked in luggage. Overall 2h later at home than planed, with really no reason, just due to incompetent people. Service level far worse then Ryanair and triple the price. Really never again. Thanks for nothing.',
 '4 Hours takeoff received Mail stating cryptic message disruptions expected limit planes leave time. So capacity Heathrow Airport hit British Airways surprise, 4h departure? Anyhow - took hour delay - forced check Hand luggage. I travel hand luggage avoid waiting ultra slow processing checked luggage. Overall

In [ ]:
def preprocessamento(texto):
  return remove_stopwords(str(texto))

In [ ]:
base.ReviewBody = base.ReviewBody.map(preprocessamento)

<ipython-input-11-4d08dbef7dd5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base.ReviewBody = base.ReviewBody.map(preprocessamento)


In [ ]:
base.ReviewBody[0]

'4 Hours takeoff received Mail stating cryptic message disruptions expected limit planes leave time. So capacity Heathrow Airport hit British Airways surprise, 4h departure? Anyhow - took hour delay - forced check Hand luggage. I travel hand luggage avoid waiting ultra slow processing checked luggage. Overall 2h later home planed, reason, incompetent people. Service level far worse Ryanair triple price. Really again. Thanks nothing.'

In [ ]:
lista_base = list(base.ReviewBody)

In [ ]:
tagged_trained_data = [TaggedDocument(words=word_tokenize(str(_d).lower()), tags=[str(i)]) for i, _d in enumerate(lista_base)]

In [ ]:
print(tagged_trained_data[0])

TaggedDocument<['4', 'hours', 'takeoff', 'received', 'mail', 'stating', 'cryptic', 'message', 'disruptions', 'expected', 'limit', 'planes', 'leave', 'time', '.', 'so', 'capacity', 'heathrow', 'airport', 'hit', 'british', 'airways', 'surprise', ',', '4h', 'departure', '?', 'anyhow', '-', 'took', 'hour', 'delay', '-', 'forced', 'check', 'hand', 'luggage', '.', 'i', 'travel', 'hand', 'luggage', 'avoid', 'waiting', 'ultra', 'slow', 'processing', 'checked', 'luggage', '.', 'overall', '2h', 'later', 'home', 'planed', ',', 'reason', ',', 'incompetent', 'people', '.', 'service', 'level', 'far', 'worse', 'ryanair', 'triple', 'price', '.', 'really', 'again', '.', 'thanks', 'nothing', '.'], ['0']>


# Doc2vec:
Instanciamos Doc2Vec com a biblioteca gensim. Usaremos dimensão 100 para representar os dados de texto.

In [ ]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=100, epochs=80)

In [ ]:
#Construindo vocabulário (corpus)
model.build_vocab(tagged_trained_data)

In [ ]:
#Treinando o modelo:
model.train(tagged_trained_data, total_examples=model.corpus_count, epochs=80)

In [ ]:
#Alguns textos semelhantes
similar_doc = model.docvecs.most_similar('0')
print(similar_doc[0])

('1093', 0.5091502666473389)


<ipython-input-19-d064e790ee58>:1: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  similar_doc = model.docvecs.most_similar('0')


In [ ]:
base.iloc[[0,1093],:]

,ReviewBody,Recommended
0,4 Hours takeoff received Mail stating cryptic ...,no
1093,Heraklion Gatwick. Left luggage belt mistake. ...,no


In [ ]:
print(model.docvecs.most_similar('1093'))

[('973', 0.6144701838493347), ('201', 0.5588767528533936), ('1406', 0.5580780506134033), ('2776', 0.545297384262085), ('2610', 0.5349730849266052), ('2434', 0.5234062075614929), ('673', 0.5234017968177795), ('342', 0.5155544877052307), ('1318', 0.5098251104354858), ('998', 0.5018484592437744)]


<ipython-input-38-71b1bcfcbb89>:1: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  print(model.docvecs.most_similar('1093'))


## Matriz de vetores:


In [ ]:
matrix = [] # <- Matriz com as 100 coordenadas de cada palavra.
for i in range(len(base.ReviewBody)):
  tokens_aux = nltk.word_tokenize(lista_base[i])
  matrix.append(model.infer_vector(tokens_aux))

In [ ]:
base_matriz = pd.DataFrame(matrix)

In [ ]:
base_matriz.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-1.183450,0.809218,-0.139340,-0.676209,-0.372331,-0.151172,-1.592105,-0.202043,-2.590573,-0.254005,...,-0.075005,0.161239,0.811026,0.398853,0.654436,2.482532,-0.491184,-0.469968,1.361894,0.212926
1,0.788173,1.504947,-0.571683,-0.815147,0.392023,-0.509446,0.426724,3.023675,-0.952043,0.509789,...,0.136950,1.060568,-3.152783,-0.216019,1.424796,1.216768,1.354456,-0.405265,1.671432,0.527812
2,-0.017948,0.223637,0.698160,-1.469464,0.075626,-0.159491,-0.160042,0.992133,-0.982238,0.199832,...,-0.134588,0.532403,0.541541,-0.268188,0.168736,-0.271838,-0.727393,0.288835,0.369698,-0.665823
3,-0.255094,-0.357623,0.424505,1.478171,0.212598,-0.349406,-0.696669,1.216174,-2.987227,0.490673,...,-1.033174,2.849348,1.784263,-0.829136,0.325805,0.205308,1.622213,0.666407,2.608713,-2.580511
4,-0.486832,1.731521,-0.397092,-0.206870,1.455745,-0.117781,0.147235,-0.059771,-0.782910,-0.420520,...,2.678159,1.301334,-0.988331,-1.299463,0.327821,1.807042,-1.581996,-0.965108,1.642586,-0.921616


In [ ]:
base_matriz.shape, base.shape

((3701, 100), (3701, 2))

In [ ]:
base_matriz.to_csv(r'matrix_100.csv')

In [ ]:
#base_matriz = pd.DataFrame('/content/matrix_100.csv')

In [ ]:
# Mudando nome das colunas para dim_
colunas = base_matriz.columns
nome_colunas = ['dim'+str(colunas[i]) for i in range(100)]
nome_colunas

In [ ]:
base_matriz.columns = nome_colunas

In [ ]:
base_matriz['classe'] = (base.Recommended == 'yes') + 0

In [ ]:
base_matriz.head()

,Unnamed: 0,dim0,dim1,dim2,dim3,dim4,dim5,dim6,dim7,dim8,...,dim91,dim92,dim93,dim94,dim95,dim96,dim97,dim98,dim99,classe
0,0,-1.183450,0.809218,-0.139340,-0.676209,-0.372331,-0.151172,-1.592105,-0.202043,-2.590573,...,0.161239,0.811026,0.398853,0.654436,2.482532,-0.491184,-0.469968,1.361894,0.212926,0
1,1,0.788173,1.504947,-0.571683,-0.815147,0.392023,-0.509446,0.426724,3.023675,-0.952043,...,1.060568,-3.152783,-0.216019,1.424796,1.216768,1.354456,-0.405265,1.671432,0.527812,0
2,2,-0.017948,0.223637,0.698160,-1.469464,0.075626,-0.159491,-0.160042,0.992133,-0.982238,...,0.532403,0.541541,-0.268188,0.168736,-0.271838,-0.727393,0.288835,0.369698,-0.665823,1
3,3,-0.255094,-0.357623,0.424505,1.478171,0.212598,-0.349406,-0.696669,1.216174,-2.987227,...,2.849348,1.784263,-0.829136,0.325805,0.205308,1.622213,0.666407,2.608713,-2.580511,0
4,4,-0.486832,1.731521,-0.397092,-0.206870,1.455745,-0.117781,0.147235,-0.059771,-0.782910,...,1.301334,-0.988331,-1.299463,0.327821,1.807041,-1.581996,-0.965108,1.642586,-0.921616,0


In [ ]:
base_matriz.to_csv(r'matrix_100_rotulada.csv')

In [ ]:
base_matriz = pd.read_csv('/content/matrix_100_rotulada.csv')

# Validação cruzada com 5fold

In [ ]:
from sklearn.model_selection import KFold, cross_val_score, cross_validate, GridSearchCV, train_test_split

In [ ]:
k_folds = KFold(n_splits = 5)

In [ ]:
X = base_matriz.iloc[:,0:100]
y = base_matriz.iloc[:,-1]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
len(y_train)

2590

# Classificador 1: Regressão logística

In [ ]:

import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score


In [ ]:
modelo_RL = LogisticRegression()

In [ ]:
scores = cross_val_score(modelo_RL, X_train, y_train, cv=5)


In [ ]:
scores.mean()

0.8471042471042469

In [ ]:
modelo_RL.fit(X_train, y_train)

LogisticRegression()

In [ ]:
y_pred_RL = modelo_RL.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_RL))

              precision    recall  f1-score   support

           0       0.88      0.88      0.88       662
           1       0.82      0.82      0.82       449

    accuracy                           0.85      1111
   macro avg       0.85      0.85      0.85      1111
weighted avg       0.85      0.85      0.85      1111



In [ ]:
accuracy_score(y_test, y_pred_RL)

0.8541854185418541

In [ ]:
confusion_matrix(y_test, y_pred_RL)

array([[582,  80],
       [ 82, 367]])

# Classificador 2: Árvore de decisão


In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
modelo_DT = DecisionTreeClassifier(random_state=0)
#modelo_2.fit(X_train, y_train)

In [ ]:
para_DT = {'criterion':['gini','entropy'],'max_depth':[4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150]}

In [ ]:
grid_DT = GridSearchCV(modelo_DT, para_DT, cv=5)

In [ ]:
grid_DT.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=0),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 20, 30,
                                       40, 50, 70, 90, 120, 150]})

In [ ]:
grid_DT.best_params_, grid_DT.best_score_

({'criterion': 'gini', 'max_depth': 5}, 0.6671814671814672)

In [ ]:
cross_val_score(modelo_DT, X_train, y_train, cv=5).mean()

0.6346872378451326

In [ ]:
modelo_DT = DecisionTreeClassifier(criterion='gini', max_depth=5)

In [ ]:
modelo_DT.fit(X_train, y_train)

In [ ]:
y_pred_DT = modelo_DT.predict(X_test)
print(classification_report(y_test, y_pred_DT))

              precision    recall  f1-score   support

           0       0.68      0.81      0.74       662
           1       0.62      0.45      0.52       449

    accuracy                           0.67      1111
   macro avg       0.65      0.63      0.63      1111
weighted avg       0.66      0.67      0.65      1111



In [ ]:
accuracy_score(y_test, y_pred_DT)

0.6651665166516652

In [ ]:
confusion_matrix(y_test, y_pred_DT)

array([[537, 125],
       [247, 202]])

# Clssificador 3: Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
modelo_RF = RandomForestClassifier()

In [ ]:
para_RF = param_grid = { 'n_estimators': [50, 100, 200, 400], 'criterion':['gini','entropy'],
              'max_depth': [None, 5, 10]}

In [ ]:
grid_RF = GridSearchCV(estimator=modelo_RF,
                           param_grid=para_RF,
                           cv=5)

In [ ]:
grid_RF.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 5, 10],
                         'n_estimators': [50, 100, 200, 400]})

In [ ]:
grid_RF.best_params_, grid_RF.best_score_

({'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 200},
 0.7996138996138996)

In [ ]:
modelo_RF = RandomForestClassifier(criterion='entropy', n_estimators=200, max_depth=10)

In [ ]:
modelo_RF.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=200)

In [ ]:
y_pred_RF = modelo_RF.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred_RF))

              precision    recall  f1-score   support

           0       0.76      0.92      0.83       662
           1       0.82      0.58      0.68       449

    accuracy                           0.78      1111
   macro avg       0.79      0.75      0.76      1111
weighted avg       0.79      0.78      0.77      1111



In [ ]:
cross_val_score(modelo_RF, X_train, y_train, cv=5).mean()

0.7988416988416989

In [ ]:
confusion_matrix(y_test, y_pred_RF)

array([[606,  56],
       [187, 262]])

# Classificador 4: KNN

In [ ]:
from pandas.core.common import random_state
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
modelo_Knn = KNeighborsClassifier()

In [ ]:
para_Knn = {'p':[1,2], 'n_neighbors':[3,5,7,9,13,21,51,61,71,81,91,101]}

In [ ]:
grid_Knn = GridSearchCV(estimator=modelo_Knn,
                           param_grid=para_Knn,
                           cv=5)

In [ ]:
grid_Knn.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [3, 5, 7, 9, 13, 21, 51, 61, 71, 81, 91,
                                         101],
                         'p': [1, 2]})

In [ ]:
grid_Knn.best_params_, grid_Knn.best_score_

({'n_neighbors': 61, 'p': 2}, 0.7181467181467183)

In [ ]:
modelo_Knn = KNeighborsClassifier(n_neighbors=61, p=2)

In [ ]:
modelo_Knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=61)

In [ ]:
cross_val_score(modelo_Knn, X, y, cv=5).mean()

0.6571094576357734

In [ ]:
y_pred_Knn = modelo_Knn.predict(X_test)
print(classification_report(y_test, y_pred_Knn))

              precision    recall  f1-score   support

           0       0.96      0.57      0.72       662
           1       0.60      0.96      0.74       449

    accuracy                           0.73      1111
   macro avg       0.78      0.77      0.73      1111
weighted avg       0.81      0.73      0.73      1111



In [ ]:
accuracy_score(y_test, y_pred_Knn)

0.729072907290729

In [ ]:
confusion_matrix(y_test, y_pred_Knn)

array([[378, 284],
       [ 17, 432]])

# Classificador 6: SVM

In [ ]:
from sklearn.svm import SVC

In [ ]:
random_state = 0
modelo_svm = SVC()

In [ ]:
parametros_svm = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}

In [ ]:
grid = GridSearchCV(modelo_svm, parametros_svm, refit = True, verbose = 3)

# fitting the model for grid search
grid.fit(X, y)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.595 total time=   0.9s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.596 total time=   0.9s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.596 total time=   0.9s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.595 total time=   0.9s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.595 total time=   0.9s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.595 total time=   0.9s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.596 total time=   0.9s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.596 total time=   0.9s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.595 total time=   0.9s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.595 total time=   0.9s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.792 total time=   0.8s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [ ]:
print("Best Hyperparameters: ", grid.best_params_)
print("Best Score: ", grid.best_score_)

Best Hyperparameters:  {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
Best Score:  0.8567859357333042


In [ ]:
modelo_svm = SVC(C=10, gamma=0.001, kernel='rbf')

In [ ]:
modelo_svm.fit(X_train, y_train)

SVC(C=10, gamma=0.001)

In [ ]:
y_pred_svm = modelo_svm.predict(X_test)#1.362
print(classification_report(y_test, y_pred_svm))

              precision    recall  f1-score   support

           0       0.88      0.88      0.88       662
           1       0.82      0.83      0.83       449

    accuracy                           0.86      1111
   macro avg       0.85      0.85      0.85      1111
weighted avg       0.86      0.86      0.86      1111



In [ ]:
accuracy_score(y_test, y_pred_svm) # A maior acurácia de todas

0.8595859585958596

In [ ]:
confusion_matrix(y_test, y_pred_svm)

array([[583,  79],
       [ 77, 372]])